## **Basic ResNet Block**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class ResNetBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride = 1):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size = 3, stride = stride, padding = 1
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(
            out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1, bias = False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut (identity or projection)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_channels, out_channels,
                    kernel_size = 1, stride = stride, bias = False
                ),
                nn.BatchNorm2d(out_channels)
            )
        # Sometimes, the convolutional layers inside the block change the data shape:
        # stride != 1: The image got smaller (e.g.,32x32 -> 16x16)
        # in_channels != out_channels: The depth increased (e.g., $64 \rightarrow 128$ filters).
        # If you tried to add the original Input (32x 32, 64 channels) to this new Output (16 x 16, 128 channels), PyTorch would crash.

    def forward(self,x):
        residual = self.shortcut(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # Adding the residual content here
        out += residual
        out = self.relu(out)

        return out


## **ResNet For CIFAR**

In [2]:
class ResNetCIFAR(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        self.in_channels = 64

        # the entrance (3 -> 64)  It defines how thick the data is ?
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        # the stage 1: 64 channels, size 32x32  (No shrinking)
        self.layer1 = nn.Sequential(
            ResNetBlock(64, 64, stride = 1),
            ResNetBlock(64, 64, stride = 1)
        )

        # the stage 2: 128 channels, size 16x16 (Shrinked!)
        self.layer2 = nn.Sequential(
            ResNetBlock(64, 128, stride = 2),
            ResNetBlock(128, 128, stride = 1)
        )

        # the stage 3: 256 channels, size 8x8 (Shrinked!)
        self.layer3 = nn.Sequential(
            ResNetBlock(128, 256, stride = 2),
            ResNetBlock(256, 256, stride = 1)
        )

        # the stage 4: 512 channels, size 4x4 (Shrinked!)
        self.layer4 = nn.Sequential(
            ResNetBlock(256,512,stride = 2),
            ResNetBlock(512,512,stride = 1)
        )

        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512, num_classes)
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Shape Flow:

# Input        → (3, 32, 32)
# Conv1        → (64, 32, 32)
# Layer1       → (64, 32, 32)
# Layer2       → (128, 16, 16)
# Layer3       → (256, 8, 8)
# Layer4       → (512, 4, 4)
# AvgPool      → (512, 1, 1)
# FC           → (10)

In [3]:
def train_CIFAR(architecture_class,optimizer_name = 'Adam',epochs = 10):
    # Data loading and transformation

    transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.4914, 0.4822, 0.4465),
            std=(0.2470, 0.2435, 0.2616)
        )
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.4914, 0.4822, 0.4465),
            std=(0.2470, 0.2435, 0.2616)
        )
    ])

    train_dataset = datasets.CIFAR10(root="./data-CIFAR",download = True, train=True, transform=transform_train)
    test_dataset  = datasets.CIFAR10(root="./data-CIFAR",download = True, train=False, transform=transform_test)

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    test_loader  = DataLoader(test_dataset, batch_size=128, shuffle=False)

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Optimizer Part

    model = architecture_class().to(device)
    criterion = nn.CrossEntropyLoss()
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    else:
        print("Unknown optimizer! Defaulting to Adam.")
        optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # ------- training loop --------

    epochs = epochs

    for epoch in range(epochs):
        model.train()
        correct = 0
        total = 0
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()

        print(f"Epoch [{epoch+1}/{epochs}] "
            f"Loss: {running_loss/len(train_loader):.4f} "
            f"Train Acc: {100*correct/total:.2f}%")
    
    # Test Evaluation Part

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()

    print(f"Test Accuracy: {100*correct/total:.2f}%")



In [5]:
train_CIFAR(architecture_class=ResNetCIFAR,optimizer_name='Adam',epochs = 20)

Epoch [1/20] Loss: 1.4959 Train Acc: 44.50%
Epoch [2/20] Loss: 1.0010 Train Acc: 64.46%
Epoch [3/20] Loss: 0.7967 Train Acc: 71.80%
Epoch [4/20] Loss: 0.6580 Train Acc: 77.23%
Epoch [5/20] Loss: 0.5623 Train Acc: 80.33%
Epoch [6/20] Loss: 0.4942 Train Acc: 82.97%
Epoch [7/20] Loss: 0.4440 Train Acc: 84.61%
Epoch [8/20] Loss: 0.3985 Train Acc: 86.26%
Epoch [9/20] Loss: 0.3656 Train Acc: 87.21%
Epoch [10/20] Loss: 0.3371 Train Acc: 88.37%
Epoch [11/20] Loss: 0.3097 Train Acc: 89.23%
Epoch [12/20] Loss: 0.2856 Train Acc: 90.15%
Epoch [13/20] Loss: 0.2634 Train Acc: 90.85%
Epoch [14/20] Loss: 0.2452 Train Acc: 91.46%
Epoch [15/20] Loss: 0.2276 Train Acc: 92.10%
Epoch [16/20] Loss: 0.2132 Train Acc: 92.54%
Epoch [17/20] Loss: 0.1968 Train Acc: 93.10%
Epoch [18/20] Loss: 0.1836 Train Acc: 93.50%
Epoch [19/20] Loss: 0.1744 Train Acc: 93.96%
Epoch [20/20] Loss: 0.1602 Train Acc: 94.30%
Test Accuracy: 89.22%
